<img src="https://raw.githubusercontent.com/Harmonize-Brazil/code-gallery/main/img/INPE_logo.png" align="left" style="height: 100px" height="100"/>
<!-- https://www.gov.br/mcti/pt-br/composicao/rede-mcti/instituto-nacional-de-pesquisas-espaciais -->
<img src="https://raw.githubusercontent.com/Harmonize-Brazil/code-gallery/main/img/harmonize_logo_without_white_background.png" align="right" style="height: 90px" height="90"/>

<h1 style="color:#274ad4; text-align: center">Vegetation Indices by City</h1>
<h4 style="color:#274ad4; text-align: center">Using the BDC datacube and SITS to obtain the NDVI/EVI mean by a Region Of Interest (ROI)</h4>
<hr style="border:2px solid #274ad4;">
<br/>  
    
<div style="text-align: center;font-size: 90%;">
    Luana Becker da Luz<sup><a href="https://orcid.org/0000-0003-2535-7658\" target="_blank" rel="noopener noreferrer"><img src="https://orcid.filecamp.com/static/thumbs/folders/qLJ1tuei4m6ugC3g.png" width="16"alt="ORCID iD" style="vertical-align: text-bottom;"/></a></sup>
    Ana Paula Dal'Asta<sup><a href="https://orcid.org/0000-0002-1286-9067" target="_blank" rel="noopener noreferrer"><img src="https://orcid.filecamp.com/static/thumbs/folders/qLJ1tuei4m6ugC3g.png" width="16"alt="ORCID iD" style="vertical-align: text-bottom;"/></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:luana.luz@inpe.br">luana.luz@inpe.br;</a><a href="mailto:apdalasta@gmail.com">apdalasta@gmail.com;</a>
    <br/><br/>
    Last Update: June 9, 2025
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter Notebook contains an example of using the Satellite Image Time Series Analysis on Earth Observation Data Cubes (<a href="https://data.inpe.br/bdc/web/en/sits-satellite-image-time-series-2" target="_blank">SITS</a>) package to access data from the Terra Moderate Resolution Imaging Spectroradiometer (MODIS) Vegetation Indices (MOD13Q1) <a href="https://data.inpe.br/bdc/stac/v1/collections/mod13q1-6.1" target="_blank">Version 6.1</a> produced by the Brazil Data Cube (BDC) project, select NDVI/EVI bands and extract mean value for each ROI municipality. In this example, we chose the Cametá/PA municipality and its neighbors.
</div>
</div>

# Initial configuration

In [ ]:
# install.packages("geobr")

library(geobr)
library(dplyr)
library(tidyr)
library(ggplot2)
library(sf)
library(sits)
library(terra)
library(glue)

# Set multicores

In [ ]:
MULTICORES <- 4L

# 1. Set ROI - Cametá and its neighbors
<hr style="border:1px solid #0077b9;">

## 1.1 Get Brasil municipalities (2022)

In [ ]:
sf_munic <- geobr::read_municipality(year = 2022)
plot(st_geometry(sf_munic))

## 1.2 Filter municipalities only for Cametá and its neighbors

In [ ]:
sf_cameta <- sf_munic %>% filter(name_muni == "Cametá")

neighbors_index <- st_intersects(sf_munic, sf_cameta, sparse = FALSE)[,1]
sf_roi <- sf_munic[neighbors_index, ]

ggplot(data = sf_roi) +
    geom_sf(fill = "lightblue") +
    geom_sf_text(aes(label = name_muni), size = 5, color = "black")

## 1.3 Obtain ROI bbox

In [ ]:
bbox <- st_bbox(sf_roi)
bbox

# 2. sits - Get data cube
<hr style="border:1px solid #0077b9;">

## 2.1 List all BDC collections

In [ ]:
sits::sits_list_collections("BDC")

## 2.2 Obtain cube based on collection, bbox, bands and dates

In [ ]:
cube <- sits::sits_cube(
    source = "BDC",
    #collection = "SENTINEL-2-16D",
    collection = "MOD13Q1-6.1",
    bands = c("NDVI", "EVI"),
    roi = bbox,
    start_date = "2020-01-04",
    end_date = "2020-12-31",
    progress = TRUE
)

sits_timeline(cube)

## 2.3 Print cube information

In [ ]:
print(cube$file_info[[1]])
print(cube$tile)
plot(cube, band = "NDVI", time = 1)

## 2.4 For each datetime, mosaic all images

**Using 4 multicores and getting data only for 2020 (23 dates), this process takes around 1.5 minutes and store 3.5GB of data in tmp folder.**

### 2.4.1 Create a tmp folder where mosaics will be computed

In [ ]:
TMP_RASTERS_FOLDER <- "tmp"

if (!file.exists(TMP_RASTERS_FOLDER)) {
    dir.create(TMP_RASTERS_FOLDER, recursive = TRUE)
    print(glue("Folder '{TMP_RASTERS_FOLDER}' was created!"))
} else{
    print(glue("Folder '{TMP_RASTERS_FOLDER}' already exists! Please, delete it using 'rm -rf' on terminal and recreate it using this cell."))
}

### 2.4.2 Runs mosaic

In [ ]:
mosaic_cube <- sits_mosaic(
    cube = cube,
    roi = bbox,
    crs = "EPSG:4326",
    output_dir = TMP_RASTERS_FOLDER,
    multicores = MULTICORES,
    progress = TRUE
)

plot(mosaic_cube, band = "NDVI", time = 1)

### 2.4.3 List generated files

In [ ]:
mosaic_filepaths <- list.files(TMP_RASTERS_FOLDER, full.names = TRUE, pattern = "*MOSAIC*")
print(length(mosaic_filepaths))
mosaic_filepaths

# 3. Extract NDVI/EVI mean for each municipality

## 3.1 Test - Extraction for only one datetime

In [ ]:
# Create raster
r <- terra::rast(
    c(file.path(TMP_RASTERS_FOLDER, "TERRA_MODIS_MOSAIC_2020-01-01_2020-01-01_EVI_v1.tif"),
      file.path(TMP_RASTERS_FOLDER, "TERRA_MODIS_MOSAIC_2020-01-01_2020-01-01_NDVI_v1.tif"))
)
# Set raster band names. Otherwise, it will set file name
names(r) <- c("EVI", "NDVI")

# Extract mean for each feature in sf_roi
df_extracted <- terra::extract(
    r,
    terra::vect(sf_roi),
    fun = mean,
    na.rm = TRUE
)
df_extracted

# Print sf_roi
head(as.data.frame(sf_roi), 10)

# Merge bands mean with ROI
sf_binded <- dplyr::bind_cols(sf_roi, df_extracted)
head(as.data.frame(sf_binded), 10)

## 3.2 Extraction for all dates and bands

### 3.2.1 Get cube files_info

In [ ]:
# Get cube file_info
file_info <- mosaic_cube["file_info"][[1]][[1]]
head(file_info)
names(file_info)

### 3.2.2 From file_info, select dates, bands and paths

In [ ]:
file_info <- file_info %>% select(c("band", "date", "path")) %>% mutate(date = as.character(date))
head(file_info)

### 3.2.3 Obtain unique dates

In [ ]:
unique_dates <- unique(file_info$date)
unique_dates

### 3.2.4 For each date, load all bands as one raster and extract mean

In [ ]:
list_sf_mean_extracted <- lapply(
    X = unique_dates,
    FUN = function(current_date) {
        # Filter cube_info to get all images from current date
        filtered <- file_info %>% filter(date == current_date)
        
        # Open all different bands files in one raster
        rasters <- terra::rast(filtered$path)
        # Rename raster names (e.g. NDVI, EVI)
        names(rasters) <- filtered$band
        
        # Extract mean in all raster bands for each ROI feature
        df_extracted <- terra::extract(
            rasters,
            terra::vect(sf_roi),
            fun = mean,
            na.rm = TRUE
        )
        
        # Bind computed df with sf_roi, remove ID column and add date column
        sf_computed <- dplyr::bind_cols(sf_roi, df_extracted) %>%
            select(-ID) %>%
            mutate(date = current_date)
        
        return (sf_computed)
    }
)
list_sf_mean_extracted[1:3]

### 3.2.5 Bind all sf dataframes into one

In [ ]:
df_municipalities_mean <- bind_rows(list_sf_mean_extracted) %>%
    as.data.frame() %>%
    pivot_longer(cols = c("NDVI", "EVI"), names_to = "band", values_to = "value") %>%
    select("code_muni", "name_muni", "date", "band", "value")
    
dim(df_municipalities_mean)
head(df_municipalities_mean, 15)

### 3.2.6 Plot 1

In [ ]:
options(repr.plot.height=5, repr.plot.width=18)

df_municipalities_mean %>%
    ggplot(aes(x=date, y=value, group=band)) +
        facet_grid(band ~ name_muni) +
        geom_line() +
        theme(text = element_text(size = 24))
        

### 3.2.7 Plot 2

In [ ]:
options(repr.plot.height=7, repr.plot.width=18)

df_municipalities_mean %>%
    mutate(value = value / 10000) %>%
    mutate(date = as.Date(date, format = "%Y-%m-%d")) %>%
    ggplot(aes(x=date, y=value, group=band)) +
        facet_grid(band ~ name_muni) +
        geom_line() +
        labs(title = "Média de EVI e NDVI por município para 2020 - MOD13Q1", x = "Data", y = "Valor") + 
        geom_point(color = "red", size = 3) + 
        scale_x_date(date_labels = "%b") + 
        theme(
            text = element_text(size = 22)
            # axis.text.x = element_text(angle = 45, hjust = 1)
        )

### 3.2.8 Plot 3

In [ ]:
options(repr.plot.height=5, repr.plot.width=18)

df_municipalities_mean %>%
    mutate(value = value / 10000) %>%
    mutate(date = as.Date(date, format = "%Y-%m-%d")) %>%
    ggplot(aes(x = date, y = value, group = band)) +
        facet_grid( ~ name_muni) +
        geom_line(aes(color = band)) +
        labs(title = "Média de EVI e NDVI por município para 2020 - MOD13Q1", x = "Data", y = "Valor") + 
        geom_point(aes(color = band), size = 3) + 
        scale_x_date(date_labels = "%b") + 
        scale_color_discrete(name = "Banda") + 
        theme(
            text = element_text(size = 22),
            legend.position = "bottom"
        )